### Import modules and things

In [130]:
from pymongo import MongoClient
import pprint
import json
import bs4
from bs4 import BeautifulSoup
import re
import numpy as np

In [462]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


### Get mongoclient

In [3]:
client = MongoClient('localhost', 27017)
db=client['reviews']
coll=db['book_reviews']

In [9]:
documents = [x for x in db['book_reviews'].find()]

In [36]:
test = documents[500]
# test is a mongo document, not the html. we need to get the 'reviews' value
# test['reviews']


In [37]:
testsoup = BeautifulSoup(test['reviews'], 'html.parser')

In [45]:
text = testsoup.find_all(class_ = "readable")

### Okay, we got to "text", sort of. Lets try to clean this up. 

In [95]:
type(text)

bs4.element.ResultSet

In [80]:
def clean_readable(readable):
    '''
    a function that takes readable, a bs4 ResultSet and turns it into a list of the strings contained in 
    '''

In [103]:
type(text[0].contents)

list

In [105]:
l = text[0].contents

In [113]:
l[0]

'\n'

In [117]:
type(l[3])

bs4.element.Tag

In [124]:
lc = [i for i in l[3].children]

In [190]:
#display different tags in the result. 


# for i in lc:
#     #print(type(i))
#     if type(i) == bs4.element.Tag:
#         print(i)
#     #if type(i) == bs4.element.NavigableString:
# #     print(i)

# get rid of all these '< br/ >', 
they're line breaks. maybe divide lines up by line breaks? no. DO NOT. that will make the nlp treat each line differently, we want an nlp document to be a whole review

In [456]:
from pymongo import MongoClient
import pprint
import json
import bs4
from bs4 import BeautifulSoup
import re
import numpy as np



def remove_text_inside_brackets(text, brackets="<>"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    return ''.join(saved_chars)


def get_text(readable):
    try:
        l = readable[0].contents # turns out there's only one readable in each review. Pretty good so far. 
    except:
        return -1
    freetextlst = [s for s in l if "freeText" in str(s)] 

    #rev_text = ' '.join(map(str, freetextlst)) includes the freetext and freetext containers, and that is..troublesome
    rev_text = str(freetextlst[-1])
    # bigstring = seperator.join(strlchildren)
    review_nlp_document = remove_text_inside_brackets(rev_text)
    
    return review_nlp_document

def str_to_rate(qual_state):
    '''
    a function that turns goodreads's "I liked it" or "I did not like it" star categories
    into the numerical 1-5 rating that they visually imply. 
    ++++++
    Attributes
    qual_state (list) a split string pulled from the beautiful soup output of .text on the rating object
    ++++++
    Returns
    user_rating (int): 1-5 score based on NUMBER OF STARS SELECTED BY THE RATER. I honestly don't understand why that's not the output in the HTML. 
    '''
    if qual_state[-3:] == ['it', 'was', 'amazing']:
        user_rating = 5
    elif qual_state[-3:] ==['really','liked','it']:
        user_rating = 4
    elif qual_state[-2:] ==['liked', 'it']:
        '''note that I belive any that include "really" will be given 4
        before we get to this elif statement, therefore we don't need 
        to worry about the issues of "really liked it" and "liked it"
        overlapping'''
        user_rating= 3
    elif qual_state[-3:]==['it','was','ok']:
        user_rating = 2
    elif qual_state[-3:]==['not','like','it']:
        user_rating = 1
    else:
        user_rating = 0
    return user_rating


def save_to_mongo(add_this, db, collection):
    '''
    Helperfunction that takes a list of reviews, appends them to dictonaries, and feeds them into mongodb
    
    Attributes:
    add_this (dict): dictonary to add to mongo
    db (str): specific string of the db to use
    collection (str): string of the collection
    '''
    #connect to the mongo thing
    client = MongoClient()
    db = client[db]
    collection = db[collection]
    
    collection.insert_many(add_this) 
    #add doc to mongodb
    client.close()

In [ ]:
if __name__ == "__main__":
    
    client = MongoClient('localhost', 27017)
    db=client['reviews']
    coll=db['book_reviews']
    documents = [x for x in db['book_reviews'].find()]
    
    ##for loop going throuhg all documents
    all_revs = []
    for i in documents:
        breakpoint = documents.index(i) ##will show me where this thing breaks

        soup = BeautifulSoup(i['reviews'], 'html.parser')
        readable = soup.find_all(class_ = "readable")
        
        
        nlp_words = get_text(readable)
        if nlp_words == -1:
            continue
            
        

        try: #some people are leaving reviews without giving the books any stars. interesting. 
            rate_string = soup.find_all(class_ = re.compile('staticStars notranslate'))[0].attrs['title']
            rate = str_to_rate(rate_string.split())
        except IndexError:
            rate = -1
        
        try: #catching the documents with nothing in them. 
            title = soup.find_all(class_ = re.compile('lightGreyText'))[0].attrs['title'], ##title is likely to produce borked results. this one is just an isbn. soo..yeah
        except IndexError:
            title = -1
            
            
        user = soup.find_all(class_ = re.compile('user'))[0].attrs['href'][11:]
        
        sub_rev = [rate, user, title, nlp_words]
        all_revs.append(sub_rev) 
        progress_bar = documents.index(i)
        update_progress(documents.index(i) / len(documents))
#         if progress_bar % 100 ==0:
#             print(progress_bar)
        
    client.close()            
    np.save("data/cleaned", all_revs) ## might need to do a mongo thing
    
    # # mongo thing GOES INSIDE THE LOOP
#         formatted_dct = {'book_id' : i['book_id']
#                          'nlp_words' : get_text(readable),
#                          'rating_given' : rate,
#                          'title' : title, ##title is likely to produce borked results. this one is just an isbn. soo..yeah
#                          'user': soup.find_all(class_ = re.compile('user'))[0].attrs['href'][11:]
#                         }

update_progress(i / number_of_elements)

update_progress(1)

Progress: [#####---------------] 22.5%


In [189]:
#ideally, replace </b> with ! to take advantage of vader tech emphasis tech

smalltext = '''This was initially going to be four stars, but the last 100/150 pages were just so 
<b>un-deniably</b>
 amazing I couldn’t help but bump it up half a star. For my first few reads they’ve started off quite light, however within the first chapter this was 
<u>dark</u>
! We got deaths and treachery, and I was 
<b>here for it/</b>'''
re.replace(r'</b>','!',smalltext)

    

In [451]:
if soup.find(class_ = re.compile('lightGreyText')) !=None:
    print(soup.find(class_ = re.compile('lightGreyText')))[0].attrs['title']
else:
    print('no')


no


In [449]:
try:
    print(soup.find(class_ = re.compile('lightGreyText')))[0].attrs['title']
except:
    print('no')

None
no


In [436]:
print(soup.find(class_ = re.compile('lightGreyText')))

None


In [434]:
#for document in documents: ##test soup
soup = BeautifulSoup(documents[breakpoint]['reviews'], 'html.parser')
soup

<div class="friendReviews elementListBrown notext">
<div class="section firstReview">
<div class="review notext" id="review_618102435" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">
<link href="https://www.goodreads.com/review/show/618102435" itemprop="url"/>
<a class="left imgcol" href="/user/show/1476929-michele" title="Michele"><img alt="Michele" src="https://images.gr-assets.com/users/1467053132p2/1476929.jpg"/></a>
<div class="left bodycol">
<div class="reviewHeader uitext stacked">
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<a class="user" href="/user/show/1476929-michele" itemprop="url" name="Michele" title="Michele">Michele</a>
</span>

        rated it
        <span class="staticStars notranslate" title="it was amazing"><span class="staticStar p10" size="15x15">it was amazing</span><span class="staticStar p10" size="15x15"></span><span class="staticStar p10" size="15x15"></span><span class="staticStar p10" size="15x15"></spa

In [ ]:

for i in documents[480:530]:
    soup = BeautifulSoup(i['reviews'], 'html.parser')
    try:
        rate_string = soup.find_all(class_ = re.compile('staticStars notranslate'))[0].attrs['title']
        rate = str_to_rate(rate_string.split())
    except IndexError:
        rate = -1
    print(documents.index(i), rate)
    

In [393]:
s = BeautifulSoup(documents[497]['reviews'], 'html.parser')

In [394]:
s

<div class="friendReviews elementListBrown">
<div class="section">
<div class="review nosyndicate" id="review_3034632747" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">
<link href="https://www.goodreads.com/review/show/3034632747" itemprop="url"/>
<a class="left imgcol" href="/user/show/103486117-dante" title="Dante"><img alt="Dante" src="https://images.gr-assets.com/users/1569898530p2/103486117.jpg"/></a>
<div class="left bodycol">
<div class="reviewHeader uitext stacked">
<a class="reviewDate createdAt right" href="/review/show/3034632747?book_show_action=false">Nov 27, 2019</a>
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<a class="user" href="/user/show/103486117-dante" itemprop="url" name="Dante" title="Dante">Dante</a>
</span>

          added it




        

          <div class="uitext greyText bookshelves">
            Shelves:
              <a class="actionLinkLite" href="/review/list/103486117-dante?shelf=magical-realism">m

In [193]:
text = testsoup.find_all(class_ = "readable")
l = text[0].contents # possible breakpoint, what is at the other text locations?
lchildren = [i for i in l[3].children]

strlchildren = [str(i) for i in lchildren]
seperator = ' '
bigstring = seperator.join(strlchildren)
review_nlp_document = remove_text_inside_brackets(bigstring)

In [194]:
print('text holds this many items: '+ str(len(text)))

text holds this many items: 1


In [323]:
def str_to_rate(qual_state):
    '''
    a function that turns goodreads's "I liked it" or "I did not like it" star categories
    into the numerical 1-5 rating that they visually imply. 
    ++++++
    Attributes
    qual_state (list) a split string pulled from the beautiful soup output of .text on the rating object
    ++++++
    Returns
    user_rating (int): 1-5 score based on NUMBER OF STARS SELECTED BY THE RATER. I honestly don't understand why that's not the output in the HTML. 
    '''
    if qual_state[-3:] == ['it', 'was', 'amazing']:
        user_rating = 5
    elif qual_state[-3:] ==['really','liked','it']:
        user_rating = 4
    elif qual_state[-2:] ==['liked', 'it']:
        '''note that I belive any that include "really" will be given 4
        before we get to this elif statement, therefore we don't need 
        to worry about the issues of "really liked it" and "liked it"
        overlapping'''
        user_rating= 3
    elif qual_state[-3:]==['it','was','ok']:
        user_rating = 2
    elif qual_state[-3:]==['not','like','it']:
        user_rating = 1
    else:
        user_rating = 0
    return user_rating


user_rating = str_to_rate(soup.find_all(class_ =re.compile('field rating'))[0].text.split())


documents[500]

{'_id': ObjectId('5e3626c4f9d4bb4f4640cb3b'),
 'book_id': '6',
 'reviews': '<div class="friendReviews elementListBrown">\n    <div class="section">\n      \n\n<div id="review_2292061646" class="review nosyndicate" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">\n  <link itemprop="url" href="https://www.goodreads.com/review/show/2292061646">\n    <a title="Beth" class="left imgcol" href="/user/show/44125660-beth"><img alt="Beth" src="https://images.gr-assets.com/users/1579985366p2/44125660.jpg"></a>\n\n  <div class="left bodycol">\n    <div class="reviewHeader uitext stacked">\n        <a class="reviewDate createdAt right" href="/review/show/2292061646?book_show_action=false">Feb 14, 2018</a>\n\n      <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">\n        <a title="Beth" class="user" itemprop="url" name="Beth" href="/user/show/44125660-beth">Beth</a>\n      </span>\n\n        rated it\n        <span class=" staticStars notranslate" title=

In [316]:
test = documents[480:500]

trythis = [get_text(i) for i in test]
    
    
    
trythis

['Oh my gosh...I ‘must’ dedicate this little review to *Cheri*....‘Queen of songs’. Songs come to Cheri when she reads.It happened to me, too. This book is DARKER than books 1-3. ( Lisa warned me) ... Characters are ‘cruel’. CRUEL ... really cruel!!! The song I started singing to myself was an old Elvis Presley song called “Don’t Be Cruel”. Indulge yourself ... sing 🎤along!!Maybe Cheri will put the link in for us to enjoy!! In this book....The Quidditch game - as cool as it is - went on too long in the beginning- Dwarves dressed up as Cupid’s to deliver Valentines to Harry was a little weird too-but .......it’s WIZARD LAND...and Harry *is* a world famous wizard.. so all is fair game in a whimsical-witchcraft-wizardry-world. Mystery, little romance some joy and sorrow, twists and turns... and always fantasy! 4.5 stars....and remember....“Don’t Be Cruel”!!!',
 "Is there anyone else out there who thinks Goblet of Fire was the high point of the whole Harry Potter series? I'm just wondering

In [311]:
all_reviews[14]

'Goblet is hand\'s down my son\'s favorite Harry Potter novel. From the Quiddich World Cup to the rebirth of Voldemort, it is nonstop action (with quite a bit of comedy thanks to Ron and others.) Our favorite characters feature prominently (and none are killed off...yet) such as, her Gandalf Dumbledore and Sirius Black. I found it entertaining and really enjoyed all the inventive ideas around magic that Rowling comes up with. I kept saying to myself, "why didn\'t I think of that?"The character development is interesting too as Harry is clearly heading into adolescence and having issues with girls. The Yuletide dance scene was absolutely the best part of the book, I thought. And, for the francophones reading this review, the French translation misses entirely the sly humorous French and Eastern European accents that Rowling uses for the Beauxbatons girls and the Durmstrang boys which were hilarious! That being said, I was a little disappointed with the Fleur Delacourt character from Bea

In [243]:
breakpoint = documents.index(i) ##will show me where this thing breaks

i = documents[500]
soup = BeautifulSoup(i['reviews'], 'html.parser')
text = soup.find_all(class_ = "readable")
l = text[0].contents # turns out there's only one readable in each review. Pretty good so far. 
freetextlst = [s for s in l if "freeText" in str(s)] 

rev_text = ' '.join(map(str, freetextlst))

# bigstring = seperator.join(strlchildren)
review_nlp_document = remove_text_inside_brackets(rev_text)
all_reviews.append(review_nlp_document)

In [284]:
freetextlst = [s for s in l if "freeText" in str(s)] 

rev_text = ' '.join(map(str, freetextlst))
rev_text

'<span id="freeTextContainer14887509396639017823"><i>”Curiosity is not a sin, but we should exercise caution with our curiosity.”</i><br/><br/>The <b>fourth</b> Buddy Read with my <a href="https://www.goodreads.com/review/show/1037220234?book_show_action=false" rel="nofollow">book lover!</a><br/><br/>This was initially going to be four stars, but the last 100/150 pages were just so <b>un-deniably</b> amazing I couldn’t help but bump it up half a star. For my first few reads they’ve started off quite light, however within the first chapter this was <u>dark</u>! We got deaths and treachery, and I was <b>here for it/</b> We got more characters and they were SO good! We had Moody, Krum, Fleur, Cedric</span> <span id="freeText14887509396639017823" style="display:none"><i>”Curiosity is not a sin, but we should exercise caution with our curiosity.”</i><br/><br/>The <b>fourth</b> Buddy Read with my <a href="https://www.goodreads.com/review/show/1037220234?book_show_action=false" rel="nofollow"

In [233]:
test = documents[:800]
lol = []
nothing = 0

for i in test:
    breakpoint = documents.index(i) ##will show me where this thing breaks
    soup = BeautifulSoup(i['reviews'], 'html.parser')
    text = soup.find_all(class_ = "readable")
    if len(text) > 1:
        print(len(text))
    if len(text)==0:
        nothing+=1


410

In [228]:

documents[488]

{'_id': ObjectId('5e3626c4f9d4bb4f4640cb2f'),
 'book_id': '6',
 'reviews': '<div class="friendReviews elementListBrown">\n    <div class="section">\n      \n\n<div id="review_447907643" class="review nosyndicate" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">\n  <link itemprop="url" href="https://www.goodreads.com/review/show/447907643">\n    <a title="Ginger" class="left imgcol" href="/user/show/14097149-ginger"><img alt="Ginger" src="https://images.gr-assets.com/users/1565278456p2/14097149.jpg"></a>\n\n  <div class="left bodycol">\n    <div class="reviewHeader uitext stacked">\n        <a class="reviewDate createdAt right" href="/review/show/447907643?book_show_action=false">Nov 03, 2012</a>\n\n      <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">\n        <a title="Ginger" class="user" itemprop="url" name="Ginger" href="/user/show/14097149-ginger">Ginger</a>\n      </span>\n\n        rated it\n        <span class=" staticStars notransl

In [318]:
small_docs = documents[480:600]

In [320]:
r = small_docs[1]

In [321]:
r['reviews']



{'_id': ObjectId('5e3626c4f9d4bb4f4640cb28'),
 'book_id': '6',
 'reviews': '<div class="friendReviews elementListBrown">\n    <div class="section">\n      \n\n<div id="review_556221106" class="review" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">\n  <link itemprop="url" href="https://www.goodreads.com/review/show/556221106">\n    <a title="Tadiana ✩Night Owl☽" class="left imgcol" href="/user/show/8734459-tadiana-night-owl"><img alt="Tadiana ✩Night Owl☽" src="https://images.gr-assets.com/users/1383845644p2/8734459.jpg"></a>\n\n  <div class="left bodycol">\n    <div class="reviewHeader uitext stacked">\n        <a class="reviewDate createdAt right" href="/review/show/556221106?book_show_action=false">Mar 08, 2013</a>\n\n      <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">\n        <a title="Tadiana ✩Night Owl☽" class="user" itemprop="url" name="Tadiana ✩Night Owl☽" href="/user/show/8734459-tadiana-night-owl">Tadiana ✩Night Owl☽</a>\n     